# Tarea 6: Transformers aplicado a traducción y clasificación de textos
## Curso: Procesamiento de Lenguaje Natural (NLP)
## Maestría en Computación
## Instituto Tecnológico de Costa Rica (ITCR)

Medio de entrega:  TecDigital.

Entregables: Un archivo jupyter con los ejercicios resueltos.

Modo de trabajo: individual o en grupos de máximo dos personas.

**Elaborado por**:


*   Victoria Orozco Arias. Carnet $2022438528$
*   Esteban Villalobos Gómez. Carnet $9913628$

**Fecha de entrega:** 15/10/2024


---



## Ejercicio 1: Creación de un transformer para traducción de textos

Este ejercicio consiste en crear de cero un modelo de transformer (codificador-decodificador) utilizando la biblioteca de PyTorch y la arquitectura básica vista en clase y que se describe en la publicación original "Attention Is All You Need" (Vaswani et al., 2017). El modelo debe ser entrenado para realizar traducciones de texto.


El dataset a utilizar está disponible en Hugging Face en librakevin/opus_books_split 1y consiste en un conjunto de datos para realizar traducción de inglés a italiano. 


El conjunto de datos es muy simple y pequeño, contiene el texto en ingles y su traducción al italiano. Ejemplo: { "en": "There was no possibility of taking a walk that day.", "it": "I. In quel giorno era impossibile passeggiare." }. Existe una versión más completa del conjunto de datos en https://huggingface.co/datasets/Helsinki-NLP/opus_books por si lo prefieren, esta incluye traducción al español.


Deben entregar un cuaderno de Jupyter con el siguiente contenido:

### 1. Carga y pre-procesamiento de los datos

  * (5 puntos) Procese el conjunto de datos (limpiar, tokenizar y realizar lo necesario
llevar a cabo el ejercicio).

In [27]:
%%capture
!pip install --upgrade transformers datasets evaluate huggingface_hub accelerate>=0.26.0 tiktoken


In [28]:
import torch
import accelerate
import pandas as pd
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForCausalLM


print(accelerate.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

1.0.1


device(type='cuda')

In [ ]:
# from huggingface_hub import login
# login()

In [29]:
splits = {'train': 'en-it/train-00000-of-00001.parquet', 'validation': 'en-it/validation-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/librakevin/opus_books_split/" + splits["train"])

In [30]:
df.head()

,translation
0,"{'en': 'Source: Project Gutenberg', 'it': 'Sou..."
1,"{'en': 'Jane Eyre', 'it': 'Jane Eyre'}"
2,"{'en': 'Charlotte Bronte', 'it': 'Charlotte Br..."
3,"{'en': 'CHAPTER I', 'it': 'PARTE PRIMA'}"
4,{'en': 'There was no possibility of taking a w...


In [31]:
df = df.drop(df.index[0])

In [32]:
df.head()

,translation
1,"{'en': 'Jane Eyre', 'it': 'Jane Eyre'}"
2,"{'en': 'Charlotte Bronte', 'it': 'Charlotte Br..."
3,"{'en': 'CHAPTER I', 'it': 'PARTE PRIMA'}"
4,{'en': 'There was no possibility of taking a w...
5,"{'en': 'We had been wandering, indeed, in the ..."


In [33]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2", model_max_length=750)

# Extract the 'en' and 'it' fields and tokenize
df['tokenized_en'] = df['translation'].apply(lambda x: tokenizer.encode(x['en'], return_tensors="pt").to(device))
df['tokenized_it'] = df['translation'].apply(lambda x: tokenizer.encode(x['it'], return_tensors="pt").to(device))

# Display the tokenized DataFrame
df.head()

,translation,tokenized_en,tokenized_it
1,"{'en': 'Jane Eyre', 'it': 'Jane Eyre'}","[[tensor(41083, device='cuda:0'), tensor(21566...","[[tensor(41083, device='cuda:0'), tensor(21566..."
2,"{'en': 'Charlotte Bronte', 'it': 'Charlotte Br...","[[tensor(24453, device='cuda:0'), tensor(11404...","[[tensor(24453, device='cuda:0'), tensor(11404..."
3,"{'en': 'CHAPTER I', 'it': 'PARTE PRIMA'}","[[tensor(41481, device='cuda:0'), tensor(314, ...","[[tensor(30709, device='cuda:0'), tensor(36, d..."
4,{'en': 'There was no possibility of taking a w...,"[[tensor(1858, device='cuda:0'), tensor(373, d...","[[tensor(40, device='cuda:0'), tensor(13, devi..."
5,"{'en': 'We had been wandering, indeed, in the ...","[[tensor(1135, device='cuda:0'), tensor(550, d...","[[tensor(14772, device='cuda:0'), tensor(23963..."


### 2. Creación del modelo

  * (20 puntos) Programe el codificador, el decodificador y todo lo necesario para implementar el transformer utilizando PyTorch.

In [34]:
# Adaptado de https://www.datacamp.com/tutorial/building-a-transformer-with-py-torch

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output
    


In [35]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [36]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [37]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [38]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [39]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [40]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)


### 3. Experimentos

 * (1 punto) Separe las muestras en datos de entrenamiento y validación.
 * (5 puntos) Entrene un modelo.
 * (2 puntos) Grafique la curva de error, explique los resultados obtenidos y ajuste el modelo o el proceso de entrenamiento apropiadamente (por ejemplo el número de épocas).
 * (2 puntos) Evalúe el modelo resultante con los datos de prueba utilizando métricas como BertScore2 y METEOR3.
 * (2 puntos) Analice los resultados y presente al menos cuatro conclusiones.
 * (2 puntos) Todas las secciones del ejercicio deben estar bien documentadas.

In [ ]:
# split df into train and validate datasets, and convert them to torch tensors. 20% of the data is reserved for validation
train_df = df.sample(frac=0.8, random_state=42)
validate_df = df.drop(train_df.index)

# convert the dataframes to torch tensors
train_en = torch.cat(train_df['tokenized_en'].tolist())
train_it = torch.cat(train_df['tokenized_it'].tolist())
validate_en = torch.cat(validate_df['tokenized_en'].tolist())
validate_it = torch.cat(validate_df['tokenized_it'].tolist())


In [ ]:
# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.685554504394531
Epoch: 2, Loss: 8.546586990356445
Epoch: 3, Loss: 8.4775972366333
Epoch: 4, Loss: 8.420077323913574
Epoch: 5, Loss: 8.364520072937012
Epoch: 6, Loss: 8.293992042541504
Epoch: 7, Loss: 8.222147941589355
Epoch: 8, Loss: 8.133575439453125
Epoch: 9, Loss: 8.048762321472168
Epoch: 10, Loss: 7.971360206604004
Epoch: 11, Loss: 7.890737056732178
Epoch: 12, Loss: 7.810125350952148
Epoch: 13, Loss: 7.72412109375
Epoch: 14, Loss: 7.633805751800537
Epoch: 15, Loss: 7.549288272857666
Epoch: 16, Loss: 7.468044281005859
Epoch: 17, Loss: 7.3885345458984375
Epoch: 18, Loss: 7.307461738586426
Epoch: 19, Loss: 7.226999759674072
Epoch: 20, Loss: 7.1388044357299805
Epoch: 21, Loss: 7.0613789558410645
Epoch: 22, Loss: 6.97879695892334
Epoch: 23, Loss: 6.909514427185059
Epoch: 24, Loss: 6.833178997039795
Epoch: 25, Loss: 6.75362491607666
Epoch: 26, Loss: 6.674020290374756
Epoch: 27, Loss: 6.602151393890381
Epoch: 28, Loss: 6.5421881675720215
Epoch: 29, Loss: 6.46768903732299

In [ ]:
transformer.eval()

# Generate random sample validation data
val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")